<a href="https://colab.research.google.com/github/KenzaxTazi/Agri-Risk/blob/master/gtc_convert_raster_to_xarray_for_spatial_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Update packages and restart runtime
%%capture
!pip install pandas==1.0.0
!pip install pandarallel
!pip install tqdm==4.42.1
!pip install affine rasterio

import os
os.kill(os.getpid(), 9)

In [0]:
#Import packages
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from affine import Affine
import os

In [0]:
#Change directory
os.chdir('/content/drive/My Drive/Team Plants')

In [0]:
#!pip install netCDF4

In [0]:
#define file paths for input (raster file) and output (netcdf file)
raster_file = '/content/drive/My Drive/Team Plants/data/soil/data_from_dataverse_resampled/raster_filename.tif'
export_netcdf_file = '/content/drive/My Drive/Team Plants/data/spatial_attributes_netcdf/netcdf_filename.nc'

In [0]:
#convert raster to xarraya and export as netcdf file
da = xr.open_rasterio(raster_file) # import raster dataset as xarray (dataarray)
da = da.rename({'x': 'lon','y': 'lat'}) # rename XY coordinates as lat and lon
da = da.reindex(lat=np.arange(89.75,-89.75-0.5,-0.5), lon=np.arange(-179.75,179.75+0.5,0.5), method='nearest') #reindex (change spatial resolution of) dataarray to be 0.5 degrees 
da.to_netcdf(export_netcdf_file) #save/export dataarray to netcdf 

END OF SCRIPT (INGONE ANYTHING BELOW THIS LINE)

In [0]:
# Compute the soil types

lats_ = xr.DataArray(list(data['nearest_lat'].values), dims='z')
lons_ = xr.DataArray(list(data['nearest_lon'].values), dims='z')

time_data = da.sel(y=lats_).sel(x=lons_)
data['soil_type'] = time_data.values[0]

In [0]:
#import climate zones raster dataset xarray
da = xr.open_rasterio('/content/drive/My Drive/Team Plants/data/climate_zones/map41/map41.tif')
transform = Affine.from_gdal(*da.attrs['transform'])
nx, ny = da.sizes['x'], da.sizes['y']
x, y = np.meshgrid(np.arange(nx)+0.5, np.arange(ny)+0.5) * transform
da.close()

da = da.reindex(y=np.arange(89.75,-89.75-0.5,-0.5), x=np.arange(-179.75,179.75+0.5,0.5), method='nearest')

da = da.rename({'x': 'lon','y': 'lat'})

print(da)

da.to_netcdf('/content/drive/My Drive/Team Plants/data/spatial_attributes_netcdf/climate_zones.nc')

In [0]:
# Compute the climate zones types

lats_ = xr.DataArray(list(data['nearest_lat'].values), dims='z')
lons_ = xr.DataArray(list(data['nearest_lon'].values), dims='z')

time_data = da.sel(lat=lats_).sel(lon=lons_)
#data['climate_zones'] = 
time_data.values[-1]

In [0]:
#to change values in netCDF4
import netCDF4
"""
dset = netCDF4.Dataset('test.nc')

dset['var'][:][dset['var'][:] < 0] = -1

dset.close()"""

"\ndset = netCDF4.Dataset('test.nc')\n\ndset['var'][:][dset['var'][:] < 0] = -1\n\ndset.close()"

In [0]:
data = pd.read_csv("/content/drive/My Drive/Team Plants/data/climate_monthly_seasonal_2005_2010_with_spatial_attributes_filtered.csv")

In [0]:
data = data[data.columns.drop(list(data.filter(regex='TX')))]
data = data[data.columns.drop(list(data.filter(regex='TN')))]
data = data[data.columns.drop(list(data.filter(regex='ID')))]
data = data[data.columns.drop(list(data.filter(regex='SU')))]
data = data[data.columns.drop(list(data.filter(regex='R20')))]
data = data[data.columns.drop(list(data.filter(regex='R10')))]
data = data[data.columns.drop(list(data.filter(regex='RR1')))]